# Tutorial 19: MCP Authentication & Security Patterns

## Overview

In this tutorial, we'll explore **authentication and security patterns** for MCP servers in enterprise environments. Building on the FastMCP server and APIM integration from previous tutorials, we'll implement production-ready authentication using **Microsoft Entra ID (Azure AD)** with **Azure API Management**.

**What you'll learn:**
- Understanding MCP authentication layers (APIM vs MCP server)
- Implementing Entra ID JWT authentication with APIM
- Acquiring tokens with MSAL for agent-to-MCP communication
- FastMCP server-side authentication options
- When to skip MCP server auth (APIM-only scenarios)
- Database integration patterns with user context (notes)

**Prerequisites:**
- Completed Tutorial 15 (FastMCP Server Basics)
- Completed Tutorial 16 (Deploy to Azure Container Apps)
- Completed Tutorial 18 (API Management Integration)
- Azure subscription with Entra ID access
- OpenAI API key or Azure OpenAI endpoint

**Architecture:**

```
┌─────────────────────────────────────────────────────────────┐
│  Authentication Flow                                        │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  User/Agent                                                 │
│      │                                                      │
│      │ 1. Acquire JWT token from Entra ID                   │
│      ▼                                                      │
│  ┌─────────────────────────────────────────────────────┐   │
│  │  Microsoft Entra ID (Azure AD)                       │   │
│  │  - Issues JWT tokens                                 │   │
│  │  - Validates user/app identity                       │   │
│  └─────────────────────────────────────────────────────┘   │
│      │                                                      │
│      │ 2. Call MCP with Bearer token                        │
│      ▼                                                      │
│  ┌─────────────────────────────────────────────────────┐   │
│  │  Azure API Management                                │   │
│  │  - validate-azure-ad-token policy                    │   │
│  │  - Rate limiting, logging                            │   │
│  │  - Forward to backend (with or without token)        │   │
│  └─────────────────────────────────────────────────────┘   │
│      │                                                      │
│      │ 3. Validated request                                 │
│      ▼                                                      │
│  ┌─────────────────────────────────────────────────────┐   │
│  │  FastMCP Server (Azure Container Apps)               │   │
│  │  - Travel tools: flights, hotels, currency           │   │
│  │  - Optional: JWTVerifier for additional validation   │   │
│  └─────────────────────────────────────────────────────┘   │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

**Time to complete:** ~45 minutes

## Part 1: Understanding MCP Authentication Layers

### Where Should Authentication Happen?

In an enterprise MCP architecture, you have **two layers** where authentication can occur:

| Layer | When to Use | Pros | Cons |
|-------|-------------|------|------|
| **APIM Only** | MCP server in private VNet, internal services, simple scenarios | Single auth point, simpler MCP server code | No user context in MCP tools |
| **APIM + MCP Server** | Multi-tenant, user-specific data, audit requirements | Full user context, defense in depth | More complex, token validation twice |

### Authentication Options

**At APIM Layer:**
- `Ocp-Apim-Subscription-Key` - Simple API key (good for dev/internal)
- `validate-azure-ad-token` - Entra ID JWT validation (recommended for production)
- `validate-jwt` - Generic JWT from any OIDC provider

**At FastMCP Layer:**
- `JWTVerifier` - Validate JWTs from Entra ID or other IdPs
- `OAuthProxy` - Full OAuth 2.0 flow with token exchange
- `StaticTokenVerifier` - Simple static token (dev/testing only)

### Recommended Pattern: APIM with Entra ID

For most enterprise scenarios, we recommend:
1. **APIM validates Entra ID tokens** - Acts as the security gateway
2. **MCP server trusts APIM** - No additional auth (runs in private network)
3. **APIM forwards user context** - Via headers like `X-User-Id`, `X-User-Email`

This gives you strong security at the perimeter while keeping MCP servers simple.

## Part 2: Setting Up Entra ID App Registration

Before we can use Entra ID authentication, we need to register applications in Azure.

### Quick Links

| Resource | URL |
|----------|-----|
| **Azure Portal - Entra ID** | [portal.azure.com → Microsoft Entra ID](https://portal.azure.com/#view/Microsoft_AAD_IAM/ActiveDirectoryMenuBlade/~/Overview) |
| **App Registrations** | [portal.azure.com → App registrations](https://portal.azure.com/#view/Microsoft_AAD_IAM/ActiveDirectoryMenuBlade/~/RegisteredApps) |

### Step 1: Register the MCP API Application

1. Go to [App registrations](https://portal.azure.com/#view/Microsoft_AAD_IAM/ActiveDirectoryMenuBlade/~/RegisteredApps)
2. Click **+ New registration**
3. Configure:
   - **Name**: `MCP Travel API`
   - **Supported account types**: Accounts in this organizational directory only
   - **Redirect URI**: Leave blank (we're using client credentials)
4. Click **Register**
5. **Copy the Application (client) ID** - this is your `MCP_API_CLIENT_ID`

### Step 2: Expose an API

1. In your MCP Travel API app registration, go to **Expose an API**
2. Click **Set** next to Application ID URI → Accept default or use `api://mcp-travel-api`
3. Click **+ Add a scope**:
   - **Scope name**: `Tools.Execute`
   - **Who can consent**: Admins and users
   - **Admin consent display name**: Execute MCP Tools
   - **Admin consent description**: Allows the app to execute MCP tools
4. Click **Add scope**

### Step 3: Register the Client Application (Agent)

1. Go back to [App registrations](https://portal.azure.com/#view/Microsoft_AAD_IAM/ActiveDirectoryMenuBlade/~/RegisteredApps)
2. Click **+ New registration**
3. **Name**: `MCP Travel Agent`
4. Click **Register**
5. **Copy the Application (client) ID** - this is your `MCP_AGENT_CLIENT_ID`
6. Go to **Certificates & secrets** → **+ New client secret**
   - **Description**: `agent-secret`
   - **Expires**: Choose appropriate expiration
7. **Copy the secret Value immediately** (you won't see it again) - this is your `MCP_AGENT_CLIENT_SECRET`
8. Go to **API permissions** → **+ Add a permission**
   - Select **My APIs** → **MCP Travel API** → Check **Tools.Execute** → **Add permissions**
9. Click **Grant admin consent for [Your Directory]**

### Step 4: Get Your Tenant ID

1. Go to [Microsoft Entra ID Overview](https://portal.azure.com/#view/Microsoft_AAD_IAM/ActiveDirectoryMenuBlade/~/Overview)
2. Copy the **Tenant ID** - this is your `AZURE_TENANT_ID`

### Step 5: Update Your `.env` File

Add these values to your `.env` file in the tutorial directory:

```bash
# Entra ID Configuration for MCP Authentication (Tutorial 19)
AZURE_TENANT_ID=xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
MCP_API_CLIENT_ID=xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
MCP_AGENT_CLIENT_ID=xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
MCP_AGENT_CLIENT_SECRET=your-client-secret-value
```

### 🔐 Secret Management Best Practices

| Environment | Approach | Details |
|-------------|----------|---------|
| **Local Development** | `.env` file | Add to `.gitignore`, never commit secrets |
| **CI/CD Pipelines** | GitHub Secrets / Azure DevOps Variables | Inject at build/deploy time |
| **Production (Azure)** | Azure Key Vault | Use Managed Identity to access secrets |
| **Container Apps** | Key Vault References | `secretref:` in container app config |

**Production Example with Key Vault:**

```python
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

# In production, use Key Vault instead of .env
credential = DefaultAzureCredential()
kv_client = SecretClient(
    vault_url="https://your-keyvault.vault.azure.net/",
    credential=credential
)

TENANT_ID = kv_client.get_secret("azure-tenant-id").value
AGENT_CLIENT_SECRET = kv_client.get_secret("mcp-agent-client-secret").value
```

**For Container Apps with Key Vault references:**
```yaml
# In your container app configuration
secrets:
  - name: mcp-agent-secret
    keyVaultUrl: https://your-keyvault.vault.azure.net/secrets/mcp-agent-client-secret
    identity: system
```

In [ ]:
# Install required packages
!pip install msal httpx python-dotenv openai azure-identity

In [ ]:
import os
import json
from typing import Dict, Any, Optional
from datetime import datetime
from dotenv import load_dotenv

# Reload environment variables (force reload)
load_dotenv(override=True)

# =============================================================================
# Configuration - Set these in your .env file (see Part 2 for setup instructions)
# =============================================================================

# Entra ID Configuration (optional - for JWT authentication)
TENANT_ID = os.getenv("AZURE_TENANT_ID", "")
API_CLIENT_ID = os.getenv("MCP_API_CLIENT_ID", "")  # MCP Travel API
AGENT_CLIENT_ID = os.getenv("MCP_AGENT_CLIENT_ID", "")  # MCP Travel Agent
AGENT_CLIENT_SECRET = os.getenv("MCP_AGENT_CLIENT_SECRET", "")

# APIM Configuration (from Tutorial 18)
APIM_GATEWAY_URL = os.getenv("APIM_GATEWAY_URL", "https://your-apim.azure-api.net")
APIM_SUBSCRIPTION_KEY = os.getenv("APIM_SUBSCRIPTION_KEY", "")

# MCP Server endpoint (via APIM)
MCP_SERVER_URL = f"{APIM_GATEWAY_URL}/travel-mcp/mcp"

# Scopes for token acquisition
API_SCOPE = f"api://{API_CLIENT_ID}/.default" if API_CLIENT_ID else ""

# =============================================================================
# Configuration Validation
# =============================================================================

def check_config():
    """Validate configuration and show status."""
    
    print("📋 Configuration Status:")
    print("=" * 60)
    
    # Check APIM configuration (required)
    print("\n🔹 APIM Configuration (Required for this tutorial):")
    apim_ok = True
    
    if APIM_GATEWAY_URL and "your-apim" not in APIM_GATEWAY_URL:
        print(f"  ✅ APIM_GATEWAY_URL: {APIM_GATEWAY_URL[:40]}...")
    else:
        print(f"  ❌ APIM_GATEWAY_URL: NOT SET")
        apim_ok = False
    
    if APIM_SUBSCRIPTION_KEY:
        print(f"  ✅ APIM_SUBSCRIPTION_KEY: {APIM_SUBSCRIPTION_KEY[:4]}...{APIM_SUBSCRIPTION_KEY[-4:]}")
    else:
        print(f"  ❌ APIM_SUBSCRIPTION_KEY: NOT SET")
        apim_ok = False
    
    # Check Entra ID configuration (optional)
    print("\n🔹 Entra ID Configuration (Optional - for JWT auth):")
    entra_configs = {
        "AZURE_TENANT_ID": TENANT_ID,
        "MCP_API_CLIENT_ID": API_CLIENT_ID,
        "MCP_AGENT_CLIENT_ID": AGENT_CLIENT_ID,
        "MCP_AGENT_CLIENT_SECRET": AGENT_CLIENT_SECRET,
    }
    
    entra_ok = True
    for name, value in entra_configs.items():
        if value and len(value) > 5:
            if "SECRET" in name:
                display = f"{value[:4]}...{value[-4:]}"
            else:
                display = f"{value[:20]}..." if len(value) > 20 else value
            print(f"  ✅ {name}: {display}")
        else:
            print(f"  ⚪ {name}: NOT SET")
            entra_ok = False
    
    print("\n" + "=" * 60)
    
    # Summary
    if apim_ok and entra_ok:
        print("✅ Full authentication configured (Entra ID + APIM)")
    elif apim_ok:
        print("✅ APIM subscription key configured")
        print("ℹ️  Entra ID not configured - will use subscription key only")
    else:
        print("❌ No authentication configured!")
        print("\n📝 Add to your .env file:")
        print("   APIM_SUBSCRIPTION_KEY=your-subscription-key")
    
    return entra_ok, apim_ok

# Run validation
ENTRA_ID_CONFIGURED, APIM_CONFIGURED = check_config()

# Also show derived values
print(f"\n📍 Derived Configuration:")
print(f"  MCP Server URL: {MCP_SERVER_URL}")
if API_SCOPE:
    print(f"  API Scope: {API_SCOPE}")

## Part 3: Configuring APIM with Entra ID Validation

### Add validate-azure-ad-token Policy to APIM

In your APIM instance, add this inbound policy to your MCP server API:

```xml
<policies>
    <inbound>
        <base />
        
        <!-- Validate Entra ID JWT Token -->
        <validate-azure-ad-token 
            tenant-id="YOUR_TENANT_ID" 
            header-name="Authorization" 
            failed-validation-httpcode="401" 
            failed-validation-error-message="Unauthorized. Access token is missing or invalid.">
            <client-application-ids>
                <application-id>YOUR_AGENT_CLIENT_ID</application-id>
            </client-application-ids>
            <audiences>
                <audience>api://mcp-travel-api</audience>
            </audiences>
        </validate-azure-ad-token>
        
        <!-- Extract user info from token and forward as headers -->
        <set-header name="X-User-Id" exists-action="override">
            <value>@(context.Request.Headers.GetValueOrDefault("Authorization","").AsJwt()?.Claims.GetValueOrDefault("oid", "unknown"))</value>
        </set-header>
        <set-header name="X-User-Email" exists-action="override">
            <value>@(context.Request.Headers.GetValueOrDefault("Authorization","").AsJwt()?.Claims.GetValueOrDefault("preferred_username", "unknown"))</value>
        </set-header>
        
    </inbound>
    <backend>
        <base />
    </backend>
    <outbound>
        <base />
    </outbound>
    <on-error>
        <base />
    </on-error>
</policies>
```

### Policy Explanation

| Element | Purpose |
|---------|---------|
| `validate-azure-ad-token` | Validates the JWT is issued by your Entra ID tenant |
| `client-application-ids` | Restricts which apps can call the API |
| `audiences` | Ensures token was issued for your API |
| `set-header` | Extracts claims and forwards to backend as headers |

This way, your FastMCP server doesn't need to validate tokens - APIM handles it!

In [ ]:
import msal
import httpx
from datetime import timedelta

class EntraIDTokenProvider:
    """
    Acquire tokens from Microsoft Entra ID using MSAL.
    
    This class handles token acquisition and caching for 
    service-to-service (client credentials) authentication.
    """
    
    def __init__(
        self,
        tenant_id: str,
        client_id: str,
        client_secret: str,
        scope: str
    ):
        self.tenant_id = tenant_id
        self.client_id = client_id
        self.scope = scope
        
        # Initialize MSAL Confidential Client
        authority = f"https://login.microsoftonline.com/{tenant_id}"
        self.app = msal.ConfidentialClientApplication(
            client_id=client_id,
            client_credential=client_secret,
            authority=authority
        )
        
        self._cached_token: Optional[Dict[str, Any]] = None
        self._token_expires_at: Optional[datetime] = None
    
    def get_token(self) -> str:
        """
        Get a valid access token, using cache if available.
        
        Returns:
            Access token string
        """
        # Check if we have a valid cached token
        if self._cached_token and self._token_expires_at:
            if datetime.now() < self._token_expires_at:
                return self._cached_token["access_token"]
        
        # Acquire new token using client credentials flow
        result = self.app.acquire_token_for_client(scopes=[self.scope])
        
        if "access_token" in result:
            self._cached_token = result
            # Token expires_in is in seconds, subtract 5 minutes buffer
            expires_in = result.get("expires_in", 3600) - 300
            self._token_expires_at = datetime.now() + timedelta(seconds=expires_in)
            
            print(f"✅ Token acquired (expires in {expires_in // 60} minutes)")
            return result["access_token"]
        else:
            error = result.get("error_description", result.get("error", "Unknown error"))
            raise Exception(f"Failed to acquire token: {error}")
    
    def get_auth_header(self) -> Dict[str, str]:
        """Get Authorization header with Bearer token."""
        return {"Authorization": f"Bearer {self.get_token()}"}


# =============================================================================
# Initialize Token Provider (only if Entra ID is configured)
# =============================================================================

token_provider = None

if ENTRA_ID_CONFIGURED:
    try:
        token_provider = EntraIDTokenProvider(
            tenant_id=TENANT_ID,
            client_id=AGENT_CLIENT_ID,
            client_secret=AGENT_CLIENT_SECRET,
            scope=API_SCOPE
        )
        print("✅ Entra ID Token Provider initialized")
        print(f"   Scope: {API_SCOPE}")
    except Exception as e:
        print(f"❌ Failed to initialize token provider: {e}")
else:
    print("⏭️  Skipping Entra ID Token Provider initialization")
    print("   Configure your .env file and re-run to enable Entra ID auth")
    print("\n💡 You can still test with APIM subscription key only (see Part 3)")

## Part 4: Authenticated MCP Client

Now let's create an MCP client that uses Entra ID authentication to call our travel tools:

In [ ]:
import uuid
import re

class AuthenticatedMCPClient:
    """
    MCP client with optional Entra ID authentication.
    
    Uses the MCP Streamable HTTP transport protocol which:
    1. Returns SSE (Server-Sent Events) format
    2. Requires session ID header for subsequent requests
    
    This client can work with:
    - Entra ID JWT tokens only
    - APIM subscription key only
    - Both (defense in depth - recommended for production)
    """
    
    def __init__(
        self,
        mcp_server_url: str,
        token_provider: Optional[EntraIDTokenProvider] = None,
        subscription_key: Optional[str] = None
    ):
        self.mcp_server_url = mcp_server_url
        self.token_provider = token_provider
        self.subscription_key = subscription_key
        self.session_id: Optional[str] = None
        
        # Validate at least one auth method is available
        if not token_provider and not subscription_key:
            print("⚠️  Warning: No authentication configured!")
            print("   Requests may fail if APIM requires authentication.")
    
    def _get_headers(self) -> Dict[str, str]:
        """Build request headers with available authentication."""
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json, text/event-stream"
        }
        
        # Add Entra ID token if available
        if self.token_provider:
            try:
                headers.update(self.token_provider.get_auth_header())
            except Exception as e:
                print(f"⚠️  Token acquisition failed: {e}")
        
        # Add subscription key if available
        if self.subscription_key:
            headers["Ocp-Apim-Subscription-Key"] = self.subscription_key
        
        # Add session ID if we have one
        if self.session_id:
            headers["Mcp-Session-Id"] = self.session_id
        
        return headers
    
    def _parse_sse_response(self, text: str) -> Dict[str, Any]:
        """Parse SSE response format and extract JSON data."""
        # SSE format is: event: message\ndata: {...}\n\n
        lines = text.strip().split('\n')
        for line in lines:
            if line.startswith('data: '):
                json_str = line[6:]  # Remove 'data: ' prefix
                return json.loads(json_str)
        return {"error": "No data found in SSE response"}
    
    async def _initialize_session(self) -> bool:
        """Initialize a new MCP session."""
        async with httpx.AsyncClient() as client:
            # Send initialize request
            response = await client.post(
                self.mcp_server_url,
                headers=self._get_headers(),
                json={
                    "jsonrpc": "2.0",
                    "id": str(uuid.uuid4()),
                    "method": "initialize",
                    "params": {
                        "protocolVersion": "2024-11-05",
                        "capabilities": {},
                        "clientInfo": {
                            "name": "AuthenticatedMCPClient",
                            "version": "1.0.0"
                        }
                    }
                },
                timeout=30.0
            )
            
            # Extract session ID from response headers
            self.session_id = response.headers.get("Mcp-Session-Id")
            
            if self.session_id:
                print(f"✅ Session established: {self.session_id[:16]}...")
                
                # Send initialized notification
                await client.post(
                    self.mcp_server_url,
                    headers=self._get_headers(),
                    json={
                        "jsonrpc": "2.0",
                        "method": "notifications/initialized",
                        "params": {}
                    },
                    timeout=30.0
                )
                return True
            else:
                print(f"❌ No session ID received. Status: {response.status_code}")
                return False
    
    async def list_tools(self) -> Dict[str, Any]:
        """List all available tools from the MCP server."""
        # Initialize session if needed
        if not self.session_id:
            if not await self._initialize_session():
                return {"error": "Failed to initialize session"}
        
        async with httpx.AsyncClient() as client:
            response = await client.post(
                self.mcp_server_url,
                headers=self._get_headers(),
                json={
                    "jsonrpc": "2.0",
                    "id": str(uuid.uuid4()),
                    "method": "tools/list",
                    "params": {}
                },
                timeout=30.0
            )
            
            if response.status_code != 200:
                return {"error": f"HTTP {response.status_code}", "message": response.text}
            
            # Parse SSE response
            return self._parse_sse_response(response.text)
    
    async def call_tool(self, tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """
        Call an MCP tool with the given arguments.
        
        Args:
            tool_name: Name of the tool to call
            arguments: Tool arguments
            
        Returns:
            Tool execution result
        """
        # Initialize session if needed
        if not self.session_id:
            if not await self._initialize_session():
                return {"error": "Failed to initialize session"}
        
        async with httpx.AsyncClient() as client:
            try:
                response = await client.post(
                    self.mcp_server_url,
                    headers=self._get_headers(),
                    json={
                        "jsonrpc": "2.0",
                        "id": str(uuid.uuid4()),
                        "method": "tools/call",
                        "params": {
                            "name": tool_name,
                            "arguments": arguments
                        }
                    },
                    timeout=30.0
                )
                
                if response.status_code == 401:
                    return {"error": "Unauthorized - check your authentication configuration"}
                elif response.status_code == 403:
                    return {"error": "Forbidden - insufficient permissions"}
                elif response.status_code != 200:
                    return {"error": f"HTTP {response.status_code}", "message": response.text}
                
                # Parse SSE response
                return self._parse_sse_response(response.text)
                
            except Exception as e:
                return {"error": str(e)}

# =============================================================================
# Initialize MCP Client with available authentication
# =============================================================================

# Determine authentication mode
auth_modes = []
if token_provider:
    auth_modes.append("Entra ID JWT")
if APIM_SUBSCRIPTION_KEY:
    auth_modes.append("APIM Subscription Key")

if auth_modes:
    mcp_client = AuthenticatedMCPClient(
        mcp_server_url=MCP_SERVER_URL,
        token_provider=token_provider,
        subscription_key=APIM_SUBSCRIPTION_KEY if APIM_SUBSCRIPTION_KEY else None
    )
    print("✅ MCP Client initialized (with SSE parsing)")
    print(f"   Server: {MCP_SERVER_URL}")
    print(f"   Auth: {' + '.join(auth_modes)}")
else:
    mcp_client = None
    print("❌ Cannot initialize MCP Client - no authentication available")
    print("\n📝 Options to fix:")
    print("   1. Configure Entra ID (see Part 2)")
    print("   2. Set APIM_SUBSCRIPTION_KEY in .env")

## Part 5: Testing Authenticated MCP Calls

Let's test our authenticated client with the FastMCP travel server tools:

In [ ]:
# Test 1: List available tools
async def test_list_tools():
    """Test listing tools with authentication."""
    print("📋 Testing MCP connection and listing tools...\n")
    
    result = await mcp_client.list_tools()
    
    if "error" in result:
        print(f"❌ Error: {result.get('error')}")
        if "message" in result:
            print(f"   Details: {result['message'][:200]}")
        return None
    
    tools = result.get("result", {}).get("tools", [])
    print(f"✅ Found {len(tools)} tools:\n")
    
    for tool in tools:
        desc = tool.get('description', 'No description')
        # Show first line of description
        first_line = desc.split('\n')[0][:60]
        print(f"  • {tool['name']}: {first_line}...")
    
    return tools

# Run the test
tools = await test_list_tools()

### Test Tool Calls with Authentication

In [ ]:
# Test 2: Call travel tools with authentication
async def test_authenticated_tool_calls():
    """Test calling MCP tools with Entra ID + APIM subscription key authentication."""
    
    print("🔐 Testing authenticated tool calls...\n")
    print("=" * 60)
    
    # Test search_flights
    print("\n✈️ Test: search_flights")
    result = await mcp_client.call_tool("search_flights", {
        "origin": "New York",
        "destination": "London",
        "departure_date": "2025-12-15"
    })
    
    if "result" in result:
        content = result["result"].get("content", [])
        for item in content:
            if item.get("type") == "text":
                try:
                    data = json.loads(item["text"])
                    print(f"  ✅ Found {len(data)} flights")
                    for flight in data[:2]:
                        print(f"     {flight['airline']} {flight['flight_number']}: ${flight['price']}")
                except:
                    print(f"  Result: {item['text'][:200]}")
    else:
        print(f"  ❌ Error: {result.get('error')}")
    
    print("\n" + "=" * 60)
    
    # Test convert_currency
    print("\n💱 Test: convert_currency")
    result = await mcp_client.call_tool("convert_currency", {
        "amount": 1000,
        "from_currency": "USD",
        "to_currency": "GBP"
    })
    
    if "result" in result:
        content = result["result"].get("content", [])
        for item in content:
            if item.get("type") == "text":
                try:
                    data = json.loads(item["text"])
                    print(f"  ✅ ${data['original_amount']} {data['original_currency']} = £{data['converted_amount']} {data['converted_currency']}")
                    print(f"     Exchange rate: {data['exchange_rate']}")
                except:
                    print(f"  Result: {item['text'][:200]}")
    else:
        print(f"  ❌ Error: {result.get('error')}")
    
    print("\n" + "=" * 60)
    
    # Test check_hotel_availability (correct param names: check_in, check_out)
    print("\n🏨 Test: check_hotel_availability")
    result = await mcp_client.call_tool("check_hotel_availability", {
        "location": "London",
        "check_in": "2025-12-15",
        "check_out": "2025-12-18",
        "guests": 2
    })
    
    if "result" in result:
        content = result["result"].get("content", [])
        for item in content:
            if item.get("type") == "text":
                try:
                    data = json.loads(item["text"])
                    print(f"  ✅ Found {len(data)} hotels")
                    for hotel in data[:2]:
                        print(f"     {hotel['hotel_name']}: ${hotel['price_per_night']}/night")
                except:
                    print(f"  Result: {item['text'][:200]}")
    else:
        print(f"  ❌ Error: {result.get('error')}")

# Run the tests
await test_authenticated_tool_calls()

## Part 6: Building an Authenticated AI Agent with Azure AI Foundry

Now let's create an AI agent using the **Azure AI Foundry Agent Service** that uses Entra ID + APIM authentication for MCP tool calls.

We'll use the `McpTool` from `azure.ai.agents.models` which supports custom headers for authentication.

In [ ]:
# Azure AI Foundry Agent Service with McpTool
# McpTool supports custom headers for Entra ID + APIM authentication

import time
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import (
    ListSortOrder,
    McpTool,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
)

# Configuration from environment
AZURE_AI_PROJECT_ENDPOINT = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
MODEL_DEPLOYMENT = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4.1")

print("=" * 70)
print("Azure AI Foundry Agent with Authenticated MCP Tools")
print("=" * 70)
print(f"""
  AI Project Endpoint: {AZURE_AI_PROJECT_ENDPOINT}
  Model Deployment: {MODEL_DEPLOYMENT}
  
  MCP Authentication:
    APIM Header: Ocp-Apim-Subscription-Key
    Entra ID Header: Authorization (Bearer token)
    MCP Server: {MCP_SERVER_URL}
""")

### Configure McpTool with Authentication Headers

In [ ]:
# Initialize McpTool with authentication headers
# This uses both APIM subscription key AND Entra ID token

# Get fresh Entra ID token (uses internal caching)
entra_token = token_provider.get_token()

# Create MCP tool pointing to APIM-protected server
mcp_tool = McpTool(
    server_label="travel_mcp_authenticated",
    server_url=MCP_SERVER_URL,
    allowed_tools=[],  # Empty list = allow all tools
)

# Add authentication headers
# 1. APIM Subscription Key (API Gateway auth)
mcp_tool.update_headers("Ocp-Apim-Subscription-Key", APIM_SUBSCRIPTION_KEY)

# 2. Entra ID Bearer Token (Application auth)
mcp_tool.update_headers("Authorization", f"Bearer {entra_token}")

print(f"""
✅ MCP Tool configured with dual authentication:
   Headers: {list(mcp_tool.headers.keys())}
   Server: {mcp_tool.server_url}
   Label: {mcp_tool.server_label}
""")

### Create AI Foundry Agent with Authenticated MCP Tools

In [ ]:
# Create AgentsClient and run authenticated agent
print("=" * 70)
print("Creating Azure AI Foundry Agent with Authenticated MCP Tools")
print("=" * 70)

# User query for testing
user_query = """
I'm planning a trip from New York to London on December 15th, 2025.
Can you help me find flights and check hotel availability for 3 nights?
Also, how much would $1000 be in British Pounds?
"""

print(f"\n[User Query]:\n{user_query}\n")

# Create the AgentsClient directly
agents_client = AgentsClient(
    endpoint=AZURE_AI_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(),
)

with agents_client:
    # Create agent with MCP tool
    agent = agents_client.create_agent(
        model=MODEL_DEPLOYMENT,
        name="Authenticated_Travel_Agent",
        instructions="""You are a helpful travel assistant with access to authenticated travel booking tools.

Your available tools (via authenticated MCP server):
- search_flights: Find flights between cities (use departure_date parameter)
- check_hotel_availability: Check hotel rooms (use check_in and check_out parameters)
- convert_currency: Convert between currencies

When helping users plan trips:
1. Search for flights using the correct date format (YYYY-MM-DD)
2. Check hotel availability for the travel dates
3. Convert currencies as needed
4. Provide a comprehensive travel summary

Be helpful, accurate, and security-conscious.""",
        tools=mcp_tool.definitions,
    )
    
    print(f"✅ Agent created: {agent.id}")
    print(f"   Using MCP Server: {mcp_tool.server_label}")
    
    # Create thread
    thread = agents_client.threads.create()
    print(f"✅ Thread created: {thread.id}")
    
    # Create message
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_query,
    )
    
    # Create run with MCP tool resources (includes auth headers)
    run = agents_client.runs.create(
        thread_id=thread.id,
        agent_id=agent.id,
        tool_resources=mcp_tool.resources
    )
    print(f"✅ Run created: {run.id}")
    
    # Process the run, handling tool approvals
    print("\n--- Processing ---")
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
        
        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break
            
            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    print(f"  🔐 Approving authenticated MCP call: {tool_call.name}")
                    tool_approvals.append(
                        ToolApproval(
                            tool_call_id=tool_call.id,
                            approve=True,
                            headers=mcp_tool.headers,  # Pass auth headers
                        )
                    )
            
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )
        
        print(f"  Status: {run.status}")
    
    print(f"\n✅ Run completed: {run.status}")
    
    if run.status == "failed":
        print(f"❌ Run failed: {run.last_error}")
    else:
        # Display messages
        messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        print("\n" + "=" * 70)
        print("CONVERSATION")
        print("=" * 70)
        for msg in messages:
            if msg.text_messages:
                role = "🧑 USER" if msg.role == "user" else "🤖 AGENT"
                last_text = msg.text_messages[-1]
                print(f"\n[{role}]:\n{last_text.text.value}")
        print("\n" + "=" * 70)
    
    # Cleanup
    agents_client.delete_agent(agent.id)
    print("✅ Agent cleaned up")

### Understanding the Authentication Flow

The diagram below shows how authentication flows through the system:

```
┌──────────────────┐     ┌───────────────────────┐     ┌──────────────────┐
│  Azure AI Agent  │     │   Azure API Mgmt      │     │  FastMCP Server  │
│  (AI Foundry)    │     │   (APIM Gateway)      │     │  (Container App) │
└────────┬─────────┘     └───────────┬───────────┘     └────────┬─────────┘
         │                           │                          │
         │  1. MCP Tool Call         │                          │
         │  + Ocp-Apim-Sub-Key       │                          │
         │  + Authorization: Bearer   │                          │
         │ ────────────────────────► │                          │
         │                           │                          │
         │                           │  2. Validate APIM Key    │
         │                           │  3. Forward + JWT Token  │
         │                           │ ────────────────────────►│
         │                           │                          │
         │                           │                          │ 4. Validate JWT
         │                           │                          │    (optional)
         │                           │                          │
         │                           │  5. MCP Response         │
         │                           │ ◄────────────────────────│
         │                           │                          │
         │  6. Tool Result           │                          │
         │ ◄──────────────────────── │                          │
         │                           │                          │
```

**Key Security Benefits:**
1. **APIM Layer**: Rate limiting, analytics, subscription management
2. **Entra ID Layer**: Identity verification, token validation, audit trail
3. **Defense in Depth**: Multiple authentication layers

In [ ]:
# Test with a second query - currency conversion focus
print("=" * 70)
print("Test 2: Currency Conversion Query")
print("=" * 70)

user_query_2 = "What is $500 USD in Euros and British Pounds?"

print(f"\n[User Query]: {user_query_2}\n")

# Get fresh token (in case previous one expired)
entra_token = token_provider.get_token()

# Create MCP tool with fresh token (restricted to currency conversion)
mcp_tool_2 = McpTool(
    server_label="travel_mcp_authenticated",
    server_url=MCP_SERVER_URL,
    allowed_tools=["convert_currency"],  # Only allow currency conversion
)
mcp_tool_2.update_headers("Ocp-Apim-Subscription-Key", APIM_SUBSCRIPTION_KEY)
mcp_tool_2.update_headers("Authorization", f"Bearer {entra_token}")

# Create new AgentsClient for this test
agents_client_2 = AgentsClient(
    endpoint=AZURE_AI_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(),
)

with agents_client_2:
    # Create agent with restricted tools
    agent = agents_client_2.create_agent(
        model=MODEL_DEPLOYMENT,
        name="Currency_Agent",
        instructions="You convert currencies. Use the convert_currency tool.",
        tools=mcp_tool_2.definitions,
    )
    
    thread = agents_client_2.threads.create()
    message = agents_client_2.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_query_2,
    )
    
    run = agents_client_2.runs.create(
        thread_id=thread.id,
        agent_id=agent.id,
        tool_resources=mcp_tool_2.resources
    )
    
    # Process run with tool approvals
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client_2.runs.get(thread_id=thread.id, run_id=run.id)
        
        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    print(f"  🔐 Authenticated call: {tool_call.name}")
                    tool_approvals.append(
                        ToolApproval(
                            tool_call_id=tool_call.id,
                            approve=True,
                            headers=mcp_tool_2.headers,
                        )
                    )
            if tool_approvals:
                agents_client_2.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )
        print(f"  Status: {run.status}")
    
    # Display result
    if run.status == "completed":
        messages = agents_client_2.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        for msg in messages:
            if msg.role == "assistant" and msg.text_messages:
                print(f"\n[Agent Response]:\n{msg.text_messages[-1].text.value}")
    else:
        print(f"❌ Run status: {run.status}")
        if run.last_error:
            print(f"   Error: {run.last_error}")
    
    agents_client_2.delete_agent(agent.id)
    print("\n✅ Test 2 complete")

### Verifying Authentication in APIM Logs

Let's query Azure Monitor / Log Analytics to see the authentication happening at the APIM layer. This shows:
- Requests hitting APIM with our authentication headers
- Successful token validation
- Request forwarding to the MCP backend

In [ ]:
# Query APIM logs and Application Insights to verify authentication flow
# This uses the Azure Monitor Query SDK to access Log Analytics and App Insights

from azure.monitor.query import LogsQueryClient
from azure.identity import DefaultAzureCredential
from datetime import timedelta

# Reload environment to get the workspace ID
load_dotenv(override=True)

# Get Log Analytics workspace ID from environment
LOG_ANALYTICS_WORKSPACE_ID = os.getenv("LOG_ANALYTICS_WORKSPACE_ID", "")

print("=" * 70)
print("Querying APIM Gateway Logs & Application Insights")
print("=" * 70)

if not LOG_ANALYTICS_WORKSPACE_ID:
    print("⚠️  LOG_ANALYTICS_WORKSPACE_ID not configured.")
else:
    print(f"📊 Workspace ID: {LOG_ANALYTICS_WORKSPACE_ID[:20]}...")
    
    try:
        # Initialize the Logs Query Client
        credential = DefaultAzureCredential()
        logs_client = LogsQueryClient(credential)
        
        # =====================================================================
        # Query 1: APIM Gateway Logs (from AzureDiagnostics table)
        # =====================================================================
        print("\n" + "=" * 70)
        print("📡 APIM Gateway Logs (AzureDiagnostics)")
        print("=" * 70)
        
        # APIM logs go to AzureDiagnostics table with Category = "GatewayLogs"
        # Column names are lowercase with _s/_d suffixes
        apim_query = """
        AzureDiagnostics
        | where ResourceProvider == "MICROSOFT.APIMANAGEMENT"
        | where Category == "GatewayLogs"
        | where TimeGenerated > ago(30m)
        | where url_s contains "travel" or apiId_s contains "travel"
        | project 
            TimeGenerated,
            httpMethod_s,
            url_s,
            httpStatus_d,
            backendResponseCode_d,
            timeTaken_d,
            ResultType,
            apimSubscriptionId_s,
            clientIP_s
        | order by TimeGenerated desc
        | take 20
        """
        
        response = logs_client.query_workspace(
            workspace_id=LOG_ANALYTICS_WORKSPACE_ID,
            query=apim_query,
            timespan=timedelta(minutes=60)
        )
        
        if response.tables and len(response.tables[0].rows) > 0:
            table = response.tables[0]
            print(f"\n✅ Found {len(table.rows)} APIM requests:\n")
            print("-" * 120)
            print(f"{'Time':<12} {'Method':<6} {'Status':<7} {'Backend':<8} {'Duration':<10} {'Result':<12} {'Subscription':<22} {'Path':<30}")
            print("-" * 120)
            
            for row in table.rows[:15]:
                time_str = str(row[0])[11:19] if row[0] else "N/A"
                method = str(row[1])[:6] if row[1] else "N/A"
                status = str(int(row[3])) if row[3] else "N/A"
                backend = str(int(row[4])) if row[4] else "-"
                duration = f"{row[5]:.0f}ms" if row[5] else "N/A"
                result = str(row[6])[:12] if row[6] else "N/A"
                sub_id = str(row[7])[:22] if row[7] else "N/A"
                url = str(row[2]) if row[2] else "N/A"
                path = url.split(".net")[-1][:30] if ".net" in url else url[-30:]
                
                # Color code results
                result_icon = "✅" if result == "Succeeded" else "❌"
                
                print(f"{time_str:<12} {method:<6} {status:<7} {backend:<8} {duration:<10} {result_icon} {result:<10} {sub_id:<22} {path:<30}")
            print("-" * 120)
        else:
            print("\nℹ️  No APIM Gateway logs found for travel-mcp API")
            print("   Logs may take 2-5 minutes to appear after enabling diagnostics")
        
        # =====================================================================
        # Query 2: Application Insights - Requests
        # =====================================================================
        print("\n" + "=" * 70)
        print("🔍 Application Insights - Requests (AppRequests)")
        print("=" * 70)
        
        appinsights_query = """
        AppRequests
        | where TimeGenerated > ago(30m)
        | where Url contains "travel" or Name contains "mcp"
        | project 
            TimeGenerated,
            Name,
            Url,
            ResultCode,
            DurationMs,
            Success,
            ClientIP
        | order by TimeGenerated desc
        | take 15
        """
        
        response2 = logs_client.query_workspace(
            workspace_id=LOG_ANALYTICS_WORKSPACE_ID,
            query=appinsights_query,
            timespan=timedelta(minutes=60)
        )
        
        if response2.tables and len(response2.tables[0].rows) > 0:
            table2 = response2.tables[0]
            print(f"\n✅ Found {len(table2.rows)} App Insights requests:\n")
            print("-" * 90)
            print(f"{'Time':<12} {'Name':<30} {'Code':<6} {'Duration':<12} {'OK':<4}")
            print("-" * 90)
            
            for row in table2.rows[:10]:
                time_str = str(row[0])[11:19] if row[0] else "N/A"
                name = str(row[1])[:30] if row[1] else "N/A"
                code = str(row[3]) if row[3] else "N/A"
                duration = f"{row[4]:.0f}ms" if row[4] else "N/A"
                success = "✅" if row[5] else "❌"
                
                print(f"{time_str:<12} {name:<30} {code:<6} {duration:<12} {success:<4}")
            print("-" * 90)
        else:
            print("\nℹ️  No App Insights requests found")
        
        # =====================================================================
        # Query 3: Application Insights - Traces (for auth details)
        # =====================================================================
        print("\n" + "=" * 70)
        print("📝 Application Insights - Traces (AppTraces)")
        print("=" * 70)
        
        traces_query = """
        AppTraces
        | where TimeGenerated > ago(30m)
        | where Message contains "travel" or Message contains "mcp" or Message contains "auth" or Message contains "token"
        | project 
            TimeGenerated,
            SeverityLevel,
            Message
        | order by TimeGenerated desc
        | take 10
        """
        
        response3 = logs_client.query_workspace(
            workspace_id=LOG_ANALYTICS_WORKSPACE_ID,
            query=traces_query,
            timespan=timedelta(minutes=60)
        )
        
        if response3.tables and len(response3.tables[0].rows) > 0:
            table3 = response3.tables[0]
            print(f"\n✅ Found {len(table3.rows)} trace messages:\n")
            
            for row in table3.rows[:5]:
                time_str = str(row[0])[11:19] if row[0] else "N/A"
                level = str(row[1]) if row[1] else "INFO"
                msg = str(row[2])[:80] if row[2] else "N/A"
                print(f"  [{time_str}] {level}: {msg}...")
        else:
            print("\nℹ️  No relevant traces found")
        
        # =====================================================================
        # Summary
        # =====================================================================
        print("\n" + "=" * 70)
        print("📋 Summary")
        print("=" * 70)
        print("""
🔐 Authentication Flow Verification:
   1. Agent acquires Entra ID JWT token (see JWT Claims cell above)
   2. Agent sends MCP request to APIM with:
      • Ocp-Apim-Subscription-Key header
      • Authorization: Bearer <jwt> header
   3. APIM validates subscription key ✓
   4. APIM validates JWT token (if policy configured) ✓
   5. APIM forwards request to MCP backend
   6. MCP server processes tool call and returns result
   
💡 Note: Logs typically take 2-5 minutes to appear in Log Analytics.
   If no logs appear, wait a few minutes and re-run this cell.
""")
            
    except Exception as e:
        print(f"❌ Error querying logs: {e}")
        print("\n💡 Ensure your Azure identity has 'Log Analytics Reader' role on the workspace.")

In [ ]:
# Alternative: Query using Azure CLI (works without Log Analytics SDK)
# This uses the az CLI to query APIM analytics

import subprocess

def query_apim_logs_via_cli():
    """Query APIM logs using Azure CLI - no SDK required."""
    
    print("=" * 70)
    print("Alternative: Query APIM Logs via Azure CLI")
    print("=" * 70)
    
    # Get APIM resource details from the gateway URL
    apim_name = APIM_GATEWAY_URL.replace("https://", "").replace(".azure-api.net", "")
    resource_group = os.getenv("APIM_RESOURCE_GROUP", "")
    
    if not resource_group:
        print("""
💡 To query APIM logs via CLI, set these environment variables:
   APIM_RESOURCE_GROUP=your-resource-group-name

Or run this Azure CLI command directly:
   az monitor metrics list \\
     --resource /subscriptions/{sub}/resourceGroups/{rg}/providers/Microsoft.ApiManagement/service/{apim} \\
     --metric "Requests" \\
     --interval PT1M \\
     --aggregation Total
""")
        return
    
    try:
        # Query recent APIM metrics
        cmd = f"""
        az monitor metrics list \
          --resource-group {resource_group} \
          --resource {apim_name} \
          --resource-type Microsoft.ApiManagement/service \
          --metric "Requests" "TotalRequests" "SuccessfulRequests" "UnauthorizedRequests" \
          --interval PT5M \
          --aggregation Total \
          --output json
        """
        
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        
        if result.returncode == 0:
            import json
            metrics = json.loads(result.stdout)
            print("\n📊 APIM Request Metrics (last 5 minutes):\n")
            for metric in metrics.get("value", []):
                name = metric.get("name", {}).get("value", "Unknown")
                timeseries = metric.get("timeseries", [])
                if timeseries and timeseries[0].get("data"):
                    latest = timeseries[0]["data"][-1]
                    total = latest.get("total", 0)
                    print(f"   {name}: {total}")
        else:
            print(f"CLI command failed: {result.stderr}")
            
    except Exception as e:
        print(f"Error: {e}")

# Run the CLI-based query
query_apim_logs_via_cli()

In [ ]:
# Decode and display the JWT token claims being sent to APIM
# This shows exactly what identity information is in the token

import base64

def decode_jwt_payload(token: str) -> dict:
    """Decode a JWT token and return its claims (without verification)."""
    try:
        # JWT has 3 parts: header.payload.signature
        parts = token.split('.')
        if len(parts) != 3:
            return {"error": "Invalid JWT format"}
        
        # Decode the payload (middle part)
        payload = parts[1]
        # Add padding if needed
        padding = 4 - len(payload) % 4
        if padding != 4:
            payload += '=' * padding
        
        decoded = base64.urlsafe_b64decode(payload)
        return json.loads(decoded)
    except Exception as e:
        return {"error": str(e)}

print("=" * 70)
print("JWT Token Claims Analysis")
print("=" * 70)

if token_provider and entra_token:
    claims = decode_jwt_payload(entra_token)
    
    if "error" not in claims:
        print(f"""
🔐 Token Identity Information:

   Application (Client):
     • App ID (azp): {claims.get('azp', 'N/A')}
     • App Name: {claims.get('app_displayname', claims.get('appid', 'N/A'))}
   
   Issuer & Audience:
     • Issuer (iss): {claims.get('iss', 'N/A')}
     • Audience (aud): {claims.get('aud', 'N/A')}
     • Tenant ID (tid): {claims.get('tid', 'N/A')}
   
   Token Validity:
     • Issued At (iat): {datetime.fromtimestamp(claims.get('iat', 0)).strftime('%Y-%m-%d %H:%M:%S') if claims.get('iat') else 'N/A'}
     • Expires (exp): {datetime.fromtimestamp(claims.get('exp', 0)).strftime('%Y-%m-%d %H:%M:%S') if claims.get('exp') else 'N/A'}
     • Not Before (nbf): {datetime.fromtimestamp(claims.get('nbf', 0)).strftime('%Y-%m-%d %H:%M:%S') if claims.get('nbf') else 'N/A'}
   
   Roles & Scopes:
     • Roles: {claims.get('roles', ['None'])}
     • Scopes (scp): {claims.get('scp', 'N/A')}

📋 This is what APIM's validate-azure-ad-token policy validates:
   ✓ Token is from the correct tenant (tid)
   ✓ Token is for our API audience (aud)  
   ✓ Token is from an allowed client app (azp)
   ✓ Token hasn't expired (exp)
""")
        
        # Show full claims for debugging
        print("\n📄 Full Token Claims (for debugging):")
        print("-" * 50)
        for key, value in sorted(claims.items()):
            if key not in ['iat', 'exp', 'nbf']:  # Skip timestamps (already shown)
                print(f"   {key}: {value}")
    else:
        print(f"❌ Could not decode token: {claims['error']}")
else:
    print("⚠️  No token available. Run the authentication cells first.")

## Part 7: FastMCP Server-Side Authentication (Optional)

For defense-in-depth or when APIM isn't available, you can add authentication directly to your FastMCP server.

### Option 1: JWTVerifier for Entra ID Tokens

```python
from fastmcp import FastMCP
from fastmcp.server.auth import JWTVerifier

# Create JWTVerifier for Entra ID
auth = JWTVerifier(
    issuer=f"https://login.microsoftonline.com/{TENANT_ID}/v2.0",
    audience=f"api://{API_CLIENT_ID}",
    # Entra ID uses JWKS for key rotation
    jwks_url=f"https://login.microsoftonline.com/{TENANT_ID}/discovery/v2.0/keys"
)

mcp = FastMCP(name="Travel Server", auth=auth)

@mcp.tool
def search_flights(origin: str, destination: str, date: str) -> dict:
    """Search flights - only accessible with valid Entra ID token."""
    return {"flights": [...]}
```

### Option 2: StaticTokenVerifier for Development

```python
from fastmcp.server.auth import StaticTokenVerifier

# Simple static token for development
auth = StaticTokenVerifier(
    token=os.getenv("MCP_DEV_TOKEN", "dev-secret-token")
)

mcp = FastMCP(name="Dev Travel Server", auth=auth)
```

### When to Use Server-Side Auth

| Scenario | Use Server-Side Auth? |
|----------|----------------------|
| MCP server behind APIM in private VNet | ❌ No - APIM handles it |
| MCP server directly exposed to internet | ✅ Yes - Required! |
| Multi-tenant with user-specific tools | ✅ Yes - Need user context |
| Development/testing | ⚠️ Optional - Use StaticTokenVerifier |

In [ ]:
# Example: Reading user context from headers (forwarded by APIM)
# This would be in your FastMCP server code

def get_user_context_example():
    """
    Example of how to read user context in FastMCP tools.
    
    When APIM validates the token and forwards user info in headers,
    your tools can access this context.
    """
    
    example_code = '''
from fastmcp import FastMCP
from fastmcp.server.dependencies import get_http_headers

mcp = FastMCP(name="User-Aware Travel Server")

@mcp.tool
async def get_my_bookings() -> dict:
    """Get bookings for the authenticated user."""
    
    # Get headers forwarded by APIM
    headers = get_http_headers()
    
    user_id = headers.get("x-user-id", "unknown")
    user_email = headers.get("x-user-email", "unknown")
    
    # Now you can query user-specific data
    bookings = await fetch_bookings_for_user(user_id)
    
    return {
        "user": user_email,
        "bookings": bookings
    }
'''
    print("Example: Accessing User Context in FastMCP Tools")
    print("=" * 60)
    print(example_code)

get_user_context_example()

## Part 8: Database Integration Patterns (Notes)

When your MCP tools need to access databases with user context, there are several patterns to consider:

### Pattern 1: Service Account (Simple)

```
Agent → APIM → MCP Server → Database (fixed credentials)
```

- MCP server uses a single service account to access the database
- Simple but no user-level data isolation
- Good for: shared data, read-only scenarios

### Pattern 2: Managed Identity (Azure-Native)

```
Agent → APIM → MCP Server → Database (Azure Managed Identity)
```

```python
from azure.identity import DefaultAzureCredential
from azure.cosmos.aio import CosmosClient

credential = DefaultAzureCredential()
client = CosmosClient(url=cosmos_url, credential=credential)
```

- No secrets to manage
- MCP server identity has database permissions
- Good for: Azure-hosted MCP servers, Cosmos DB, Azure SQL

### Pattern 3: Token Passthrough (User Context)

```
Agent (user token) → APIM → MCP Server → Database (with user context)
```

```python
@mcp.tool
async def get_user_data() -> dict:
    headers = get_http_headers()
    user_id = headers.get("x-user-id")
    
    # Query with user filter (Row-Level Security)
    items = await container.query_items(
        query="SELECT * FROM c WHERE c.userId = @userId",
        parameters=[{"name": "@userId", "value": user_id}]
    )
    return {"data": items}
```

- APIM extracts user ID from JWT and forwards in header
- MCP tool uses user ID to filter data
- Good for: Multi-tenant apps, user-specific data

### Pattern 4: On-Behalf-Of Flow (Full Delegation)

```
Agent (user token) → APIM → MCP Server → Exchange token → Database
```

- MCP server exchanges user token for database token
- Full audit trail with user identity
- Most complex but most secure
- Good for: Enterprise scenarios with strict compliance

### Recommended: Cosmos DB with User Scoping

For Azure Cosmos DB (aligned with your data modeling best practices):

```python
from azure.cosmos.aio import CosmosClient
from azure.identity.aio import DefaultAzureCredential

@mcp.tool
async def get_travel_history(user_id: str = None) -> dict:
    """Get travel history with user scoping."""
    
    # Get user from APIM-forwarded header if not provided
    if not user_id:
        headers = get_http_headers()
        user_id = headers.get("x-user-id")
    
    # Use partition key for efficient queries
    container = cosmos_client.get_database_client("travel").get_container_client("bookings")
    
    items = container.query_items(
        query="SELECT * FROM c WHERE c.userId = @userId ORDER BY c.createdAt DESC",
        parameters=[{"name": "@userId", "value": user_id}],
        partition_key=user_id  # Efficient partition-scoped query
    )
    
    return {"history": [item async for item in items]}
```

In [ ]:
# Demonstration: Complete authentication flow summary

def print_auth_flow_summary():
    """Print a summary of the authentication flow we've implemented."""
    
    summary = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                    MCP Authentication Flow Summary                           ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  1️⃣  AGENT ACQUIRES TOKEN                                                    ║
║     ┌─────────────────────────────────────────────────────────────────────┐ ║
║     │  token_provider = EntraIDTokenProvider(                            │ ║
║     │      tenant_id=TENANT_ID,                                          │ ║
║     │      client_id=AGENT_CLIENT_ID,                                    │ ║
║     │      client_secret=AGENT_CLIENT_SECRET,                            │ ║
║     │      scope="api://mcp-travel-api/.default"                         │ ║
║     │  )                                                                  │ ║
║     │  token = token_provider.get_token()                                │ ║
║     └─────────────────────────────────────────────────────────────────────┘ ║
║                                        │                                     ║
║                                        ▼                                     ║
║  2️⃣  AGENT CALLS MCP WITH BEARER TOKEN                                       ║
║     ┌─────────────────────────────────────────────────────────────────────┐ ║
║     │  Headers:                                                           │ ║
║     │    Authorization: Bearer <jwt_token>                               │ ║
║     │    Content-Type: application/json                                   │ ║
║     └─────────────────────────────────────────────────────────────────────┘ ║
║                                        │                                     ║
║                                        ▼                                     ║
║  3️⃣  APIM VALIDATES TOKEN                                                    ║
║     ┌─────────────────────────────────────────────────────────────────────┐ ║
║     │  <validate-azure-ad-token tenant-id="..." >                        │ ║
║     │    <audiences><audience>api://mcp-travel-api</audience></audiences>│ ║
║     │  </validate-azure-ad-token>                                        │ ║
║     │                                                                     │ ║
║     │  <set-header name="X-User-Id">                                     │ ║
║     │    <value>@(jwt.Claims["oid"])</value>                             │ ║
║     │  </set-header>                                                      │ ║
║     └─────────────────────────────────────────────────────────────────────┘ ║
║                                        │                                     ║
║                                        ▼                                     ║
║  4️⃣  MCP SERVER RECEIVES VALIDATED REQUEST                                   ║
║     ┌─────────────────────────────────────────────────────────────────────┐ ║
║     │  Headers available in FastMCP:                                     │ ║
║     │    X-User-Id: "user-object-id"                                     │ ║
║     │    X-User-Email: "user@example.com"                                │ ║
║     │                                                                     │ ║
║     │  Tools can use this context for user-specific operations           │ ║
║     └─────────────────────────────────────────────────────────────────────┘ ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""
    print(summary)

print_auth_flow_summary()

## Summary

In this tutorial, we explored **MCP Authentication & Security Patterns** for enterprise deployments:

### What We Covered

1. **Multi-Layer Authentication Architecture**
   - APIM subscription keys for API management
   - Entra ID JWT tokens for identity-based access
   - Understanding when each layer validates

2. **Token Acquisition with MSAL**
   - Using `ConfidentialClientApplication` for service-to-service auth
   - Acquiring tokens with proper scopes
   - Token caching and refresh strategies

3. **APIM Integration Patterns**
   - Subscription key header (`Ocp-Apim-Subscription-Key`)
   - `validate-azure-ad-token` policy for Entra ID
   - Extracting claims and passing context to backend

4. **Agent Authentication Integration**
   - Wrapping MCP calls with proper authentication
   - Tool execution with authenticated context
   - Passing user identity through the call chain

5. **FastMCP Server-Side Authentication**
   - JWTVerifier for production deployments
   - Configuring audience and issuer validation
   - Development vs production auth modes

### Key Takeaways

- **Defense in Depth**: Use both APIM (subscription keys) and identity tokens (Entra ID) for comprehensive security
- **Token Validation at the Edge**: Let APIM handle JWT validation to offload work from your MCP server
- **Context Propagation**: Pass user identity through headers so MCP tools can make user-specific decisions
- **Separation of Concerns**: APIM handles auth/throttling, MCP server handles business logic

### Next Steps

- Explore database integration with user-scoped access patterns
- Implement row-level security based on JWT claims
- Add audit logging for compliance requirements
- Consider Managed Identity for Azure-to-Azure authentication

### Related Resources

- [FastMCP Authentication Documentation](https://gofastmcp.com/servers/auth)
- [Azure APIM JWT Validation](https://learn.microsoft.com/en-us/azure/api-management/validate-azure-ad-token-policy)
- [MSAL Python Documentation](https://learn.microsoft.com/en-us/entra/msal/python/)
- [Tutorial 18: APIM MCP Integration](18_apim_mcp_integration.ipynb)